In [1]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import sys
print(sys.path)
FOLDERNAME = 'finalproject'
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

%cd /content/drive/My\Drive/$FOLDERNAME

print(sys.path)
%load_ext autoreload
%autoreload 2

Mounted at /content/drive
['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython']
/content/drive/MyDrive/finalproject
['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/My Drive/finalproject']


In [2]:
!pip install transformers
!pip install datasets
!pip install positional-encodings
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from transformers import BertModel
import json
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
import os
import torch
import torch.nn as nn
from torch import Tensor
import math
from torch.nn import functional as F
import copy
from typing import Optional, List

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import numpy as np
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [4]:
from our_transformer import Transformer
from our_detr import DETR
import pandas as pd

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# **Loading dataset, Preprocessing**

In [5]:
'''import pandas as pd
from pandas.io.json import json_normalize
with open("task1_headline_ABSA_train.json") as json_file:
  dataset = json.load(json_file) 
data = pd.json_normalize(dataset.values()).apply(pd.Series.explode)
print(data.head(10))
#print(json.dumps(dataset, indent = 4))
data = pd.DataFrame(data)
data1= pd.DataFrame(data.pop('info').values.tolist())
data.reset_index(inplace=True, drop=True)
data_final = pd.concat([data, data1], axis=1, join="inner")
data_final.to_csv('data_final.csv', index = False)'''

'import pandas as pd\nfrom pandas.io.json import json_normalize\nwith open("task1_headline_ABSA_train.json") as json_file:\n  dataset = json.load(json_file) \ndata = pd.json_normalize(dataset.values()).apply(pd.Series.explode)\nprint(data.head(10))\n#print(json.dumps(dataset, indent = 4))\ndata = pd.DataFrame(data)\ndata1= pd.DataFrame(data.pop(\'info\').values.tolist())\ndata.reset_index(inplace=True, drop=True)\ndata_final = pd.concat([data, data1], axis=1, join="inner")\ndata_final.to_csv(\'data_final.csv\', index = False)'

# **Extract triplet (target, aspect, sentiment)**

In [6]:
import csv
data_final= pd.read_csv("data_final.csv")
target = data_final.loc[:,"target"]
sentiment = data_final.loc[:,"sentiment_score"]
aspect = data_final.loc[:,"aspects"]
import re
aspect1 = []
for aspect in aspect:
  aspect = re.sub('[/!@#$]', ' ', aspect[2:])
  aspect1.append(aspect.split(" ")[0])
triplet = []
for i in range(len(aspect1)):
  triplet.append((target[i],aspect1[i],sentiment[i]))  
print(triplet)

[('Royal Mail', 'Corporate', -0.374), ('AstraZeneca', 'Corporate', -0.24), ('Morrisons', 'Corporate', -0.161), ('Insight', 'Corporate', 0.137), ('Primark', 'Corporate', 0.704), ('Pearson', 'Stock', 0.446), ('TalkTalk', 'Corporate', 0.136), ('Tesco', 'Corporate', 0.192), ('Unilever', 'Corporate', 0.213), ('Tesco', 'Stock', 0.389), ('Royal Dutch Shell', 'Corporate', 0.65), ('Shire', 'Corporate', 0.195), ('China Merchants Group', 'Corporate', 0.119), ('AstraZeneca', 'Corporate', 0.242), ('Teva', 'Corporate', 0.476), ('EasyJet', 'Corporate', 0.329), ('Blinkbox', 'Corporate', -0.195), ('Tesco', 'Corporate', -0.335), ('Shire', 'Corporate', 0.332), ('Ocwen', 'Corporate', 0.171), ('Morrisons', 'Corporate', -0.113), ('Blinkbox', 'Stock', -0.357), ('Tesco', 'Stock', -0.389), ('Shire', 'Corporate', 0.235), ('Omnis Pharmaceuticals', 'Corporate', 0.295), ('Shire', 'Corporate', 0.291), ('Thetrainline.com', 'Stock', 0.245), ('IAG', 'Corporate', 0.12), ('Tesco', 'Stock', 0.719), ('Holcim Lafarge', 'St

# **Create dataset for target aspect prediction**

In [7]:
targets = pd.DataFrame(triplet, columns = ['target', 'aspect', 'sentiment'])


# Get Bert and put preprocessed data in




In [8]:
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
for param in model.parameters():
    param.requires_grad = False

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# INPUT TEXT -> WORD EMBEDDING 


In [9]:
# Tokenize all of the sentences and map the tokens to their word IDs.
input_ids = []
attention_masks = []
# get sentences from the preprocessed data.
sentences = data_final.loc[:,"sentence"]
# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 32,           # Pad & truncate all sentences.
                        padding='max_length',
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences)
print('Token IDs:', input_ids)

Original:  0      Royal Mail chairman Donald Brydon set to step ...
1      Stakes High for AstraZeneca Heart Drug Facing ...
2      UPDATE 1-Dairy Crest loses a third of Morrison...
3      Insight hires Aviva's David Hillier for multi-...
4      Primark racks up a happy Christmas after stron...
                             ...                        
493    Aviva, M&G suspend property funds as investors...
494    UK housing market steadies after Brexit dip, P...
495    BRIEF-Aviva aims to increase dividend pay-out ...
496       Builder Persimmon hails 6% rise in house sales
497    EasyJet attracts more passengers in June but s...
Name: sentence, Length: 498, dtype: object
Token IDs: tensor([[  101,  2548,  5653,  ...,     0,     0,     0],
        [  101,  7533,  2152,  ...,     0,     0,     0],
        [  101, 10651,  1015,  ...,     0,     0,     0],
        ...,
        [  101,  4766,  1011,  ...,     0,     0,     0],
        [  101, 12508,  2566,  ...,     0,     0,     0],
     

# **Get positional encoding to put into decoder**

In [10]:
from positional_encodings import PositionalEncoding1D
n = np.zeros(shape=(498, 32, 768))
x = torch.tensor(n, dtype=torch.float32)
print(x.dtype)
p_enc_1d_model = PositionalEncoding1D(768)
p_enc = p_enc_1d_model(x)
print(p_enc.dtype)

torch.float32
torch.float32


# ***Making FULL transformer model with Bert & decoder ***

# **Initialize Transformer**

In [11]:
my_model = Transformer(d_model=768, nhead=8, num_decoder_layers=6, dim_feedforward=2048, dropout=0.1,
                activation="relu", normalize_before=False,
                return_intermediate_dec=False)

# **Initialize DETR with my model**

In [12]:
#총 데이터셋에서 aspect의 종류는 사실상 4개 밖에 없다.
print(len(targets['aspect'].unique()))
#총 데이터셋에서 target의 종류는 226개가 있다
print(len(targets['target'].unique()))

4
226


# **DETR 만들기**

In [13]:
my_detr = DETR(my_model, p_enc,4, 226, 32)
x = my_detr.forward(input_ids, attention_masks)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1933: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [20]:
print(x['pred_sentiment'])

tensor([[[ 0.0445],
         [ 0.0570],
         [ 0.0302],
         ...,
         [ 0.0077],
         [ 0.1511],
         [-0.0107]],

        [[-0.0721],
         [-0.1204],
         [-0.0075],
         ...,
         [-0.0275],
         [ 0.0263],
         [ 0.0641]],

        [[-0.1626],
         [ 0.0422],
         [-0.0548],
         ...,
         [-0.0366],
         [-0.0370],
         [ 0.0620]],

        ...,

        [[ 0.0971],
         [ 0.0433],
         [-0.0042],
         ...,
         [-0.1054],
         [-0.0093],
         [ 0.0322]],

        [[ 0.0946],
         [-0.0378],
         [ 0.0255],
         ...,
         [ 0.0329],
         [-0.0467],
         [-0.1323]],

        [[ 0.0362],
         [ 0.1377],
         [ 0.1290],
         ...,
         [-0.0134],
         [ 0.0564],
         [ 0.0778]]], grad_fn=<SelectBackward0>)


In [15]:
class SetCriterion(nn.Module):
    """ This class computes the loss for DETR.
    The process happens in two steps:
        1) we compute hungarian assignment between ground truth sand the outputs of the model
        2) we supervise each pair of matched ground-truth / prediction (supervise class and box)
    """
    def __init__(self, num_targets, num_aspects, matcher, weight_dict, eos_coef1, eos_coef2, losses):
        """ Create the criterion.
        Parameters:
            num_targets: number of target categories, omitting the special no-object category
            num_aspects: number of aspect categories
            matcher: module able to compute a matching between targets and proposals
            weight_dict: dict containing as key the names of the losses and as values their relative weight.
            eos_coef: relative classification weight applied to the no-object category
            losses: list of all the losses to be applied. See get_loss for list of available losses.
        """
        super().__init__()
        self.num_targets = num_targets
        self.num_aspects = num_aspects
        self.matcher = matcher
        self.weight_dict = weight_dict
        self.eos_coef1 = eos_coef1
        self.eos_coef2 = eos_coef2
        self.losses = losses
        empty_weight1 = torch.ones(self.num_targets + 1)
        empty_weight2 = torch.ones(self.num_aspects + 1)
        empty_weight1[-1] = self.eos_coef1
        empty_weight2[-1] = self.eos_coef2
        self.register_buffer('empty_weight1', empty_weight1)
        self.register_buffer('empty_weight2', empty_weight2)

    def loss_targets(self, outputs, targets, indices, num_boxes, log=True):
        """Classification loss (NLL)
        targets dicts must contain the key "targets" containing a tensor of dim [nb_targets]
        """
        assert 'pred_logits' in outputs
        src_logits = outputs['pred_target']

        idx = self._get_src_permutation_idx(indices)
        target_classes_o = torch.cat([t["target"][J] for t, (_, J) in zip(targets, indices)])
        target_classes = torch.full(src_logits.shape[:2], self.num_classes,
                                    dtype=torch.int64, device=src_logits.device)
        target_classes[idx] = target_classes_o

        loss_ce = F.cross_entropy(src_logits.transpose(1, 2), target_classes, self.empty_weight1)
        losses = {'loss_ce': loss_ce}

        if log:
            # TODO this should probably be a separate loss, not hacked in this one here
            losses['class_error'] = 100 - accuracy(src_logits[idx], target_classes_o)[0]
        return losses
        
    def loss_aspects(self, outputs, targets, indices, num_boxes, log=True):
        """Classification loss (NLL)
        targets dicts must contain the key "aspects" containing a tensor of dim [nb_targets]
        """
        assert 'pred_aspect' in outputs
        src_logits = outputs['pred_aspect']

        idx = self._get_src_permutation_idx(indices)
        target_classes_o = torch.cat([t["aspect"][J] for t, (_, J) in zip(aspects, indices)])
        target_classes = torch.full(src_logits.shape[:2], self.num_classes,
                                    dtype=torch.int64, device=src_logits.device)
        target_classes[idx] = target_classes_o

        loss_ce = F.cross_entropy(src_logits.transpose(1, 2), target_classes, self.empty_weight)
        losses = {'loss_ce': loss_ce}

        if log:
            # TODO this should probably be a separate loss, not hacked in this one here
            losses['class_error'] = 100 - accuracy(src_logits[idx], target_classes_o)[0]
        return losses

    def loss_sentiment(self, outputs, targets, indices, num_boxes):
        """Compute the losses related to the bounding boxes, the L1 regression loss and the GIoU loss
           targets dicts must contain the key "boxes" containing a tensor of dim [nb_target_boxes, 4]
           The target boxes are expected in format (center_x, center_y, w, h), normalized by the image size.
        """
        assert 'pred_sentiment' in outputs
        idx = self._get_src_permutation_idx(indices)
        src_sentiment = outputs['pred_sentiment'][idx]
        target_sentiment = torch.cat([t['sentiment'][i] for t, (_, i) in zip(targets, indices)], dim=0)
        loss_sentiment = F.mse_loss(src_sentiment, target_sentiment, reduction='none')

        losses = {}
        losses['sentiment'] = loss_sentiment

        return losses

    def _get_src_permutation_idx(self, indices):
        # permute predictions following indices
        batch_idx = torch.cat([torch.full_like(src, i) for i, (src, _) in enumerate(indices)])
        src_idx = torch.cat([src for (src, _) in indices])
        return batch_idx, src_idx

    def _get_tgt_permutation_idx(self, indices):
        # permute targets following indices
        batch_idx = torch.cat([torch.full_like(tgt, i) for i, (_, tgt) in enumerate(indices)])
        tgt_idx = torch.cat([tgt for (_, tgt) in indices])
        return batch_idx, tgt_idx

    def get_loss(self, loss, outputs, targets, indices, **kwargs):
        loss_map = {
            'target': self.loss_target,
            'aspect': self.loss_aspect,
            'sentiment': self.loss_sentiment,
        }
        assert loss in loss_map, f'do you really want to compute {loss} loss?'
        return loss_map[loss](outputs, targets, indices, **kwargs)

    def forward(self, outputs, targets):
        """ This performs the loss computation.
        Parameters:
             outputs: dict of tensors, see the output specification of the model for the format
             targets: list of dicts, such that len(targets) == batch_size.
                      The expected keys in each dict depends on the losses applied, see each loss' doc
        """
        outputs_without_aux = {k: v for k, v in outputs.items() if k != 'aux_outputs'}

        # Retrieve the matching between the outputs of the last layer and the targets
        indices = self.matcher(outputs_without_aux, targets)
        
        # Compute all the requested losses
        losses = {}
        for loss in self.losses:
            losses.update(self.get_loss(loss, outputs, targets, indices))

        # In case of auxiliary losses, we repeat this process with the output of each intermediate layer.
        if 'aux_outputs' in outputs:
            for i, aux_outputs in enumerate(outputs['aux_outputs']):
                indices = self.matcher(aux_outputs, targets)
                for loss in self.losses:
                    if loss == 'masks':
                        # Intermediate masks losses are too costly to compute, we ignore them.
                        continue
                    kwargs = {}
                    if loss == 'labels':
                        # Logging is enabled only for the last layer
                        kwargs = {'log': False}
                    l_dict = self.get_loss(loss, aux_outputs, targets, indices, **kwargs)
                    l_dict = {k + f'_{i}': v for k, v in l_dict.items()}
                    losses.update(l_dict)

        return losses



# **Hungarian matcher(미완성)**

In [16]:
class SetCriterion(nn.Module):
    """ This class computes the loss for DETR.
    The process happens in two steps:
        1) we compute hungarian assignment between ground truth sand the outputs of the model
        2) we supervise each pair of matched ground-truth / prediction (supervise class and box)
    """
    def __init__(self, num_targets, num_aspects, matcher, weight_dict, eos_coef1, eos_coef2, losses):
        """ Create the criterion.
        Parameters:
            num_targets: number of target categories, omitting the special no-object category
            num_aspects: number of aspect categories
            matcher: module able to compute a matching between targets and proposals
            weight_dict: dict containing as key the names of the losses and as values their relative weight.
            eos_coef: relative classification weight applied to the no-object category
            losses: list of all the losses to be applied. See get_loss for list of available losses.
        """
        super().__init__()
        self.num_targets = num_targets
        self.num_aspects = num_aspects
        self.matcher = matcher
        self.weight_dict = weight_dict
        self.eos_coef1 = eos_coef1
        self.eos_coef2 = eos_coef2
        self.losses = losses
        empty_weight1 = torch.ones(self.num_targets + 1)
        empty_weight2 = torch.ones(self.num_aspects + 1)
        empty_weight1[-1] = self.eos_coef1
        empty_weight2[-1] = self.eos_coef2
        self.register_buffer('empty_weight1', empty_weight1)
        self.register_buffer('empty_weight2', empty_weight2)

    def loss_targets(self, outputs, targets, indices, num_boxes, log=True):
        """Classification loss (NLL)
        targets dicts must contain the key "targets" containing a tensor of dim [nb_targets]
        """
        assert 'pred_logits' in outputs
        src_logits = outputs['pred_target']

        idx = self._get_src_permutation_idx(indices)
        target_classes_o = torch.cat([t["target"][J] for t, (_, J) in zip(targets, indices)])
        target_classes = torch.full(src_logits.shape[:2], self.num_classes,
                                    dtype=torch.int64, device=src_logits.device)
        target_classes[idx] = target_classes_o

        loss_ce = F.cross_entropy(src_logits.transpose(1, 2), target_classes, self.empty_weight1)
        losses = {'loss_ce': loss_ce}

        if log:
            # TODO this should probably be a separate loss, not hacked in this one here
            losses['class_error'] = 100 - accuracy(src_logits[idx], target_classes_o)[0]
        return losses
        
    def loss_aspects(self, outputs, targets, indices, num_boxes, log=True):
        """Classification loss (NLL)
        targets dicts must contain the key "aspects" containing a tensor of dim [nb_targets]
        """
        assert 'pred_aspect' in outputs
        src_logits = outputs['pred_aspect']

        idx = self._get_src_permutation_idx(indices)
        target_classes_o = torch.cat([t["aspect"][J] for t, (_, J) in zip(aspects, indices)])
        target_classes = torch.full(src_logits.shape[:2], self.num_classes,
                                    dtype=torch.int64, device=src_logits.device)
        target_classes[idx] = target_classes_o

        loss_ce = F.cross_entropy(src_logits.transpose(1, 2), target_classes, self.empty_weight)
        losses = {'loss_ce': loss_ce}

        if log:
            # TODO this should probably be a separate loss, not hacked in this one here
            losses['class_error'] = 100 - accuracy(src_logits[idx], target_classes_o)[0]
        return losses

    def loss_sentiment(self, outputs, targets, indices, num_boxes):
        """Compute the losses related to the bounding boxes, the L1 regression loss and the GIoU loss
           targets dicts must contain the key "boxes" containing a tensor of dim [nb_target_boxes, 4]
           The target boxes are expected in format (center_x, center_y, w, h), normalized by the image size.
        """
        assert 'pred_sentiment' in outputs
        idx = self._get_src_permutation_idx(indices)
        src_sentiment = outputs['pred_sentiment'][idx]
        target_sentiment = torch.cat([t['sentiment'][i] for t, (_, i) in zip(targets, indices)], dim=0)
        loss_sentiment = F.mse_loss(src_sentiment, target_sentiment, reduction='none')

        losses = {}
        losses['sentiment'] = loss_sentiment

        return losses

    def _get_src_permutation_idx(self, indices):
        # permute predictions following indices
        batch_idx = torch.cat([torch.full_like(src, i) for i, (src, _) in enumerate(indices)])
        src_idx = torch.cat([src for (src, _) in indices])
        return batch_idx, src_idx

    def _get_tgt_permutation_idx(self, indices):
        # permute targets following indices
        batch_idx = torch.cat([torch.full_like(tgt, i) for i, (_, tgt) in enumerate(indices)])
        tgt_idx = torch.cat([tgt for (_, tgt) in indices])
        return batch_idx, tgt_idx

    def get_loss(self, loss, outputs, targets, indices, **kwargs):
        loss_map = {
            'target': self.loss_target,
            'aspect': self.loss_aspect,
            'sentiment': self.loss_sentiment,
        }
        assert loss in loss_map, f'do you really want to compute {loss} loss?'
        return loss_map[loss](outputs, targets, indices, **kwargs)

    def forward(self, outputs, targets):
        """ This performs the loss computation.
        Parameters:
             outputs: dict of tensors, see the output specification of the model for the format
             targets: list of dicts, such that len(targets) == batch_size.
                      The expected keys in each dict depends on the losses applied, see each loss' doc
        """
        outputs_without_aux = {k: v for k, v in outputs.items() if k != 'aux_outputs'}

        # Retrieve the matching between the outputs of the last layer and the targets
        indices = self.matcher(outputs_without_aux, targets)
        
        # Compute all the requested losses
        losses = {}
        for loss in self.losses:
            losses.update(self.get_loss(loss, outputs, targets, indices, num_boxes))

        # In case of auxiliary losses, we repeat this process with the output of each intermediate layer.
        if 'aux_outputs' in outputs:
            for i, aux_outputs in enumerate(outputs['aux_outputs']):
                indices = self.matcher(aux_outputs, targets)
                for loss in self.losses:
                    if loss == 'masks':
                        # Intermediate masks losses are too costly to compute, we ignore them.
                        continue
                    kwargs = {}
                    if loss == 'labels':
                        # Logging is enabled only for the last layer
                        kwargs = {'log': False}
                    l_dict = self.get_loss(loss, aux_outputs, targets, indices, num_boxes, **kwargs)
                    l_dict = {k + f'_{i}': v for k, v in l_dict.items()}
                    losses.update(l_dict)

        return losses



In [17]:
x['pred_target'].shape
x['pred_aspect'].shape

torch.Size([498, 32, 5])

# **Target data **

In [18]:
targets

,target,aspect,sentiment
0,Royal Mail,Corporate,-0.374
1,AstraZeneca,Corporate,-0.240
2,Morrisons,Corporate,-0.161
3,Insight,Corporate,0.137
4,Primark,Corporate,0.704
...,...,...,...
493,M&G,Corporate,-0.807
494,Perssimon,Market,0.339
495,Aviva,Corporate,0.439
496,Persimmon,Corporate,0.435
